<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/utilities/examples_MLFlow_InColab_with_NGROK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import our helper Library

In [ ]:
%load_ext autoreload
%autoreload 2

!pip install mlflow --quiet
!pip install pyngrok --quiet

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# add the path to the python library path
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks")

Start our MLFlow server in the background and expose the endpoint using nGROk

In [ ]:
import ColabLibrary

url_mlflow=ColabLibrary.startMLFlowUI()
print (url_mlflow)

In [ ]:
import mlflow

mlflow.set_tracking_uri(uri=url_mlflow)

# Create a new MLflow Experiment
mlflow.set_experiment("ColabLibrary example")

Now we can just implement a quick small test.Lets train the wellknown IRIS model:


In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
   #  "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
model = LogisticRegression(**params)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)



Now we can upload the metrics and the model to mlflow

In [ ]:
from mlflow.models import infer_signature

mlflow.set_experiment("IRIS Logistic Regression")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="Iris-Dataset-Logistic-Regression-test",
    )